<a href="https://colab.research.google.com/github/pranavrajaram/datascience/blob/main/MLB_rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting the MLB world Series
The MLB is currently ongoing. The simplest way to predict a winner in the Fall, could be to check who is on top of the current standings. This, however, wouldn't take into account the strength of each team's schedule. We'll use the methods discussed in class instead. 

For this project, you are given a dataset containing all the games played this season through June. 

What you have to do:

*   Practice manipulating the dataset using Pandas. Check the introductory notebook `intro_to_pandas,ipynb`
*   Create the Massey matrix 
*   Solve the corresponding system to rank the teams using Massey's method 
*   Re-do the steps for the Colley method
*   Discuss any differences in the results, and explain why that happened. 
*   Pick a winner!


In [67]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/RodericGuigoCorominas/datascience/main/datasets/mlb_june_2022.csv'
df = pd.read_csv(url)
print(df)

             home    visitor  score home  score visitor
0            Cubs    Brewers           5              4
1          Royals  Guardians           3              1
2       Cardinals    Pirates           9              0
3       Nationals       Mets           1              5
4          Braves       Reds           3              6
..            ...        ...         ...            ...
732       Rangers       Rays           3              0
733          Cubs    Brewers           8              7
734     Athletics     Astros           1              3
735  Diamondbacks     Braves           8              7
736       Dodgers    Pirates           3              5

[737 rows x 4 columns]


In [69]:
# the code snippet below may be useful
all_teams = np.unique(df[['home','visitor']].to_numpy().flatten())
num_teams = len(all_teams)

print("There are",num_teams,"teams")
print(all_teams)

There are 30 teams
['Angels' 'Astros' 'Athletics' 'Blue Jays' 'Braves' 'Brewers' 'Cardinals'
 'Cubs' 'Diamondbacks' 'Dodgers' 'Giants' 'Guardians' 'Mariners' 'Marlins'
 'Mets' 'Nationals' 'Orioles' 'Padres' 'Phillies' 'Pirates' 'Rangers'
 'Rays' 'Red Sox' 'Reds' 'Rockies' 'Royals' 'Tigers' 'Twins' 'White Sox'
 'Yankees']


In [63]:
# tip: it may be useful to use a dictionary to translate between teams and indices. This way, you can use the name of a team to represent the corresponding row or column
teams_dict = dict(zip(all_teams, range(num_teams)))

# 'Rangers' now corresponds to the index 20
teams_dict['Rangers']

20

In [83]:
# your code...

home_games = df.groupby(['home']).count().reset_index()
home_games.columns = ['team','home_games','a','b']
home_games = home_games[['team','home_games']]

away_games = df.groupby(['visitor']).count().reset_index()
away_games.columns = ['team','away_games','a','b']
away_games = away_games[['team','away_games']]

combined_gp = pd.merge(home_games, away_games, on = ["team","team"])
combined_gp['total_gp'] = combined_gp['home_games'] + combined_gp['away_games']
combined_gp = combined_gp[['team','total_gp']]


home_df = df.groupby(['home']).sum().reset_index()

home_df.columns = ['team','home_pf','home_pa']

away_df = df.groupby(['visitor']).sum().reset_index()

away_df.columns = ['team','away_pa','away_pf']

combined_df = pd.merge(home_df, away_df, on = ["team","team"])

combined_df['pf'] = combined_df['home_pf'] + combined_df['away_pf']
combined_df['pa'] = combined_df['home_pa'] + combined_df['away_pa']
combined_df = combined_df[['team','pf','pa']]
combined_df['diff'] = combined_df['pf'] - combined_df['pa']

final = pd.merge(combined_gp, combined_df, on = ['team','team'])

print(final)



            team  total_gp   pf   pa  diff
0         Angels        50  236  202    34
1         Astros        50  201  158    43
2      Athletics        52  175  221   -46
3      Blue Jays        48  191  191     0
4         Braves        50  207  223   -16
5        Brewers        51  235  196    39
6      Cardinals        49  232  188    44
7           Cubs        49  216  223    -7
8   Diamondbacks        51  203  232   -29
9        Dodgers        49  270  154   116
10        Giants        48  244  222    22
11     Guardians        45  208  189    19
12      Mariners        49  201  210    -9
13       Marlins        46  184  180     4
14          Mets        51  266  194    72
15     Nationals        51  206  277   -71
16       Orioles        51  190  233   -43
17        Padres        49  205  182    23
18      Phillies        50  224  224     0
19       Pirates        48  166  247   -81
20       Rangers        48  211  198    13
21          Rays        49  203  196     7
22       Re